## Assignment 3
Haider, Shabih, 21027325

In [10]:
import numpy as np
import pandas as pd
import wbgapi as wb
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster as cluster


In [2]:
# This function reads the data file and manipulates dataframe into our desired format
def world_bank_dfParser(df_file_name):
    df = pd.read_csv(df_file_name, index_col=[0,1])
    
    # Returning 2 Data Frames, One with years as columns, other with countries as columns
    return df, df.T   

In [3]:
# This function retrieves data from world bank website and saves it into a file
def get_world_bank_data(c_codes):
    # Indicator ids
    indicator_ids = ['EG.USE.PCAP.KG.OE', 'EN.ATM.METH.KT.CE', 'EN.ATM.GHGT.KT.CE']
    print(wb.series.info(indicator_ids))
    
    # Fetching data from World bank for listed countries and indicators
    wb_dataframe = wb.data.DataFrame(indicator_ids, country_codes, time=range(2010,2015)) # retrieving data for 2010 - 2015
    
    # Renaming indicator Ids with actual names
    wb_dataframe = wb_dataframe.rename(index={
        'EG.USE.PCAP.KG.OE' : 'Energy use', 
        'EN.ATM.METH.KT.CE' : 'Methane emissions', 
        'EN.ATM.GHGT.KT.CE' : 'Total greenhouse gas emissions'
    })
    
    # Renaming country and indicator column names
    wb_dataframe.index.rename(['Country', 'Indicators'], inplace=True)
    
    # Checking for null values in dataframe
    display(wb_dataframe.isnull().sum())
    
    # Writing received data to csv file
    print('Writing data to csv file.......')
    wb_dataframe.to_csv('wb_dataframe.csv', index=True)
    print('Done')
    


In [4]:
# Selected countries
country_codes = ['CHN', 'FRA', 'DEU', 'IND', 'USA', 'WLD']
country_names = {
    "CHN" : "China",
    "FRA" : "France",
    "DEU" : "Germany",
    "IND" : "India",
    "USA" : "United States",
    "WLD" : "World"
}

# This following function is used to fetch the data from WB API. Only call this function if you want to update the data.
get_world_bank_data(country_codes)

df_1, df_2 = world_bank_dfParser('wb_dataframe.csv')

id                 value
-----------------  -----------------------------------------------------
EG.USE.PCAP.KG.OE  Energy use (kg of oil equivalent per capita)
EN.ATM.METH.KT.CE  Methane emissions (kt of CO2 equivalent)
EN.ATM.GHGT.KT.CE  Total greenhouse gas emissions (kt of CO2 equivalent)
                   3 elements


YR2010    0
YR2011    0
YR2012    0
YR2013    0
YR2014    0
dtype: int64

Writing data to csv file.......
Done


In [18]:
df_2

Country             CHN                                                   \
Indicators   Energy use Total greenhouse gas emissions Methane emissions   
YR2010      1954.722556                       0.000000          0.000000   
YR2011      2085.083022                       0.524066          0.297514   
YR2012      2149.602569                       0.719842          0.612937   
YR2013      2204.243299                       0.993512          0.802009   
YR2014      2224.354898                       1.000000          1.000000   

Country           DEU                                                   \
Indicators Energy use Total greenhouse gas emissions Methane emissions   
YR2010       1.000000                       0.934923          1.000000   
YR2011       0.415198                       0.220815          0.554348   
YR2012       0.447970                       0.571535          0.746377   
YR2013       0.735546                       1.000000          0.416667   
YR2014       0.000000                       0.000000          0.000000   

Country           FRA                                                   \
Indicators Energy use Total greenhouse gas emissions Methane emissions   
YR2010       1.000000                       1.000000          1.000000   
YR2011       0.525448                       0.629661          0.515738   
YR2012       0.496334                       0.689884          0.302663   
YR2013       0.487607                       0.709816          0.210654   
YR2014       0.000000                       0.000000          0.000000   

Country           IND                                                   \
Indicators Energy use Total greenhouse gas emissions Methane emissions   
YR2010       0.000000                       0.000000          0.000000   
YR2011       0.218118                       0.220918          0.977337   
YR2012       0.500574                       0.515557          0.076487   
YR2013       0.589183                       0.643441          0.960340   
YR2014       1.000000                       1.000000          1.000000   

Country           USA                                                   \
Indicators Energy use Total greenhouse gas emissions Methane emissions   
YR2010       1.000000                       1.000000          1.000000   
YR2011       0.545707                       0.518508          0.312677   
YR2012       0.000000                       0.000000          0.000000   
YR2013       0.116004                       0.295311          0.008179   
YR2014       0.306347                       0.338300          0.083674   

Country           WLD                                                   
Indicators Energy use Total greenhouse gas emissions Methane emissions  
YR2010       0.000000                       0.000000          0.000000  
YR2011       0.150436                       0.459635          0.405315  
YR2012       0.375937                       0.673468          0.653795  
YR2013       0.429133                       0.921906          0.695214  
YR2014       1.000000                       1.000000          1.000000

In [5]:
# Some statistical information about the data. Statistical analysis is performed on all years.
df_2.groupby(['Country', 'Indicators'], axis = 1).describe()

count          mean           std  \
Country Indicators                                                          
CHN     Energy use                        5.0  2.123601e+03  1.087821e+02   
        Total greenhouse gas emissions    5.0  1.127961e+07  6.869141e+05   
        Methane emissions                 5.0  1.141070e+06  5.677093e+04   
DEU     Energy use                        5.0  3.892567e+03  8.167410e+01   
        Total greenhouse gas emissions    5.0  8.656200e+05  1.757072e+04   
        Methane emissions                 5.0  5.866000e+04  1.033755e+03   
FRA     Energy use                        5.0  3.838640e+03  1.265881e+02   
        Total greenhouse gas emissions    5.0  4.524000e+05  1.715740e+04   
        Methane emissions                 5.0  6.353600e+04  1.570694e+03   
IND     Energy use                        5.0  5.962339e+02  2.852880e+01   
        Total greenhouse gas emissions    5.0  2.794820e+06  1.870530e+05   
        Methane emissions                 5.0  6.522680e+05  1.822531e+03   
USA     Energy use                        5.0  6.985938e+03  1.148929e+02   
        Total greenhouse gas emissions    5.0  6.172692e+06  1.747729e+05   
        Methane emissions                 5.0  6.270800e+05  1.339614e+04   
WLD     Energy use                        5.0  1.892388e+03  1.731195e+01   
        Total greenhouse gas emissions    5.0  4.341875e+07  1.054904e+06   
        Methane emissions                 5.0  7.843902e+06  9.457410e+04   

                                                 min           25%  \
Country Indicators                                                   
CHN     Energy use                      1.954723e+03  2.085083e+03   
        Total greenhouse gas emissions  1.020382e+07  1.107455e+07   
        Methane emissions               1.063830e+06  1.106190e+06   
DEU     Energy use                      3.779462e+03  3.869816e+03   
        Total greenhouse gas emissions  8.436600e+05  8.525500e+05   
        Methane emissions               5.716000e+04  5.831000e+04   
FRA     Energy use                      3.659088e+03  3.833534e+03   
        Total greenhouse gas emissions  4.241300e+05  4.535100e+05   
        Methane emissions               6.186000e+04  6.273000e+04   
IND     Energy use                      5.616534e+02  5.779944e+02   
        Total greenhouse gas emissions  2.564130e+06  2.671200e+06   
        Methane emissions               6.501400e+05  6.504100e+05   
USA     Energy use                      6.872027e+03  6.905599e+03   
        Total greenhouse gas emissions  5.968740e+06  6.108670e+06   
        Methane emissions               6.181500e+05  6.184100e+05   
WLD     Energy use                      1.874658e+03  1.881478e+03   
        Total greenhouse gas emissions  4.181750e+07  4.302206e+07   
        Methane emissions               7.704390e+06  7.807040e+06   

                                                 50%           75%  \
Country Indicators                                                   
CHN     Energy use                      2.149603e+03  2.204243e+03   
        Total greenhouse gas emissions  1.139983e+07  1.185453e+07   
        Methane emissions               1.151100e+06  1.178020e+06   
DEU     Energy use                      3.876948e+03  3.939530e+03   
        Total greenhouse gas emissions  8.666700e+05  8.813000e+05   
        Methane emissions               5.869000e+04  5.922000e+04   
FRA     Energy use                      3.836656e+03  3.847072e+03   
        Total greenhouse gas emissions  4.563200e+05  4.572500e+05   
        Methane emissions               6.311000e+04  6.399000e+04   
IND     Energy use                      5.991556e+02  6.057940e+02   
        Total greenhouse gas emissions  2.814000e+06  2.875980e+06   
        Methane emissions               6.535300e+05  6.535900e+05   
USA     Energy use                      6.960684e+03  7.029955e+03   
        Total greenhouse gas emissions  6.129040

In [11]:
def norm(array):
    min_val = np.min(array)
    max_val = np.max(array)
    
    scaled = (array-min_val) / (max_val-min_val)
    
    return scaled

def norm_df(df):
    # iterate over all columns
    for col in df.columns[1:]:     # excluding the first column
        df[col] = norm(df[col])
        
    return df

In [14]:
# normalise result and inspect results
normalised_df = norm_df(df_2)
print(normalised_df.describe())
print()


Country             CHN                                                   \
Indicators   Energy use Total greenhouse gas emissions Methane emissions   
count          5.000000                       5.000000          5.000000   
mean        2123.601269                       0.647484          0.542492   
std          108.782091                       0.413433          0.398728   
min         1954.722556                       0.000000          0.000000   
25%         2085.083022                       0.524066          0.297514   
50%         2149.602569                       0.719842          0.612937   
75%         2204.243299                       0.993512          0.802009   
max         2224.354898                       1.000000          1.000000   

Country           DEU                                                   \
Indicators Energy use Total greenhouse gas emissions Methane emissions   
count        5.000000                       5.000000          5.000000   
mean         0.51

In [15]:
def makeplot(df, col1, col2):
    plt.figure(figsize=(5.0,5.0))
    plt.plot(df[col1], df[col2], "o", markersize=3)
    
    plt.xlabel(col1)
    plt.ylabel(col2)
    plt.show()
    
    
# exploratory plots
makeplot(df_2, "Energy use", "Total greenhouse gas emissions")
makeplot(df_2, "Methane emissions", "Total greenhouse gas emissions")

KeyError: 'Energy use'

<Figure size 360x360 with 0 Axes>